In [6]:
import torch
from tensorflow.keras.datasets import fashion_mnist

In [13]:
# load the data
(x_train_full, y_train_full), (x_test, y_test) = fashion_mnist.load_data()

# split train full into train and valid, and normalise so input is float between 0 and 1
x_valid, x_train = x_train_full[:5000] / 255.0, x_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# manually type out the class names 
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# extract input dims
dims = x_train[0].shape

In [38]:
torch.as_tensor(y_train)

tensor([4, 0, 7,  ..., 3, 0, 5], dtype=torch.uint8)

In [43]:
# can make the model using toch.nn module

# model structure
m = torch.nn.Sequential(
        torch.nn.Flatten(),
        torch.nn.Linear(784, 300),
        torch.nn.ReLU(),
        torch.nn.Linear(300, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, len(class_names)),
        torch.nn.Softmax()
    )

# loss function
loss = torch.nn.CrossEntropyLoss(reduction='mean')

# function to yield minibatches
def data_generator(x, y, b):
    """
    generates minibatch from data set x,y of size b. Assumes instances are in the first dimension
    """
    num_batches = int(x.shape[0] / b)
    for k in range(num_batches):
        start, end = k * b, (k + 1) * b
        yield torch.as_tensor(x[start:end]), torch.as_tensor(y[start:end])

# function to train using simple SGD
def sgd(model, loss_fn, x, y, batch_size, epochs, learning_rate):
    """
    Perform simple gradient descent 
    """
    
    for i in range(epochs):
        print(f"Starting epoch number: {i+1}")
    
        for j, (x_, y_) in enumerate(data_generator(x, y, batch_size)):
            print("!!")
            y_hat = model(x_)
            l = loss_fn(torch.argmax(y_hat,1), y_)
            model.zero_grad()
            l.backward()

            with torch.no_grad():
                for param in model.paramaters():
                    param -= learning_rate * param.grad
        
        print(f"Epoch {i+1} complete, current loss value: {l.item()}")
    
    return model
            

In [44]:
tm = sgd(m, loss, x_train[:64], y_train[:64], 16, 1, 0.1)

Starting epoch number: 1
!!


RuntimeError: expected scalar type Float but found Double